# Classification Report Notebook
In this report, Performance of every model on test dataset is compared

In [1]:
# Import Necessary Packages
import numpy as np
import pandas as pd
import nltk
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from utils.helper_functions import read_dataframes, tokenize, encode_sequence, collate_fn
import torch
from train_lstm import TextDataset
from models.lstm import LSTM
from torch.utils.data import DataLoader

In [22]:
train_df, val_df, test_df  = read_dataframes()

# Machine Learning Models

In [23]:
# Load Label Encoder 
with open('model_files/class_names.pkl', 'rb') as f:
    lb = pickle.load(f)

In [24]:
# Load Count Vectorizer
with open('model_files/count_vectorizer.pkl', 'rb') as f:
    count_vec = pickle.load(f)

In [25]:
models_dict = [
    'logistic_regression', 'multinomial_naive_bayes',
    'svm_with_kernal', 'random_forest', 'xgboost'
]

In [6]:
x_test_bow = np.asarray(count_vec.transform(test_df['product_title']).todense())
y_test  = test_df['category'].to_numpy()

In [7]:
for model_name in models_dict:
    with open(f'model_files/{model_name}_classification.pkl', 'rb') as f:
        model = pickle.load(f)
    y_pred = model.predict(x_test_bow)
    print(f'Model Name : {model_name}')
    print(classification_report(y_true=y_test, y_pred=y_pred, target_names=lb.classes_.tolist()))

Model Name : logistic_regression
                  precision    recall  f1-score   support

            CPUs       1.00      1.00      1.00       773
 Digital Cameras       1.00      1.00      1.00       540
     Dishwashers       0.94      0.98      0.96       685
        Freezers       0.99      0.93      0.96       442
 Fridge Freezers       0.97      0.96      0.96      1100
         Fridges       0.94      0.93      0.94       717
      Microwaves       0.99      0.96      0.97       468
   Mobile Phones       0.99      1.00      0.99       816
             TVs       0.96      0.99      0.98       713
Washing Machines       0.97      0.97      0.97       809

        accuracy                           0.97      7063
       macro avg       0.97      0.97      0.97      7063
    weighted avg       0.97      0.97      0.97      7063

Model Name : multinomial_naive_bayes
                  precision    recall  f1-score   support

            CPUs       1.00      1.00      1.00       77

# LSTM Model 

In [41]:
from create_vocab import CreateVocab

In [42]:
vocab = CreateVocab(first_run=False, load_embeddings=False)

In [43]:
test_df['tokenized_x'] = test_df['product_title'].apply(lambda x : tokenize(x))

In [44]:
test_df['encoded_x'] = test_df['tokenized_x'].apply(lambda x : encode_sequence(x, vocab.word2idx))

In [45]:
test_dataset = TextDataset(test_df[['encoded_x', 'tokenized_x', 'category']])

In [46]:
test_dataloder = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn, pin_memory=True)

In [47]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [76]:
model = LSTM(len(vocab.word2idx.keys()), vocab.embedding_dim, None, 100, num_output=10)

In [77]:
model_dict = torch.load('model_files/lstm/epoch_4.tar')

In [78]:
model.load_state_dict(model_dict)

<All keys matched successfully>

In [79]:
y_true = []
y_preds = []

In [52]:
for i, (text, text_lengths, labels) in enumerate(test_dataloder):
    y_true.append(labels)
    text = text.to(device)
    outputs = model(text, text_lengths)
    _, preds = torch.max(outputs, 1)
    y_preds.append(preds.detach().cpu())

In [53]:
y_preds = torch.tensor([item for sublist in y_preds for item in sublist])
y_true = torch.tensor([item for sublist in y_true for item in sublist])

In [54]:
print(classification_report(y_true=y_true, y_pred=y_preds, target_names=lb.classes_.tolist()))

                  precision    recall  f1-score   support

            CPUs       1.00      1.00      1.00       773
 Digital Cameras       1.00      1.00      1.00       540
     Dishwashers       0.94      0.96      0.95       685
        Freezers       0.95      0.94      0.94       442
 Fridge Freezers       0.95      0.95      0.95      1100
         Fridges       0.93      0.92      0.93       717
      Microwaves       0.97      0.96      0.97       468
   Mobile Phones       0.99      0.99      0.99       816
             TVs       0.98      0.98      0.98       713
Washing Machines       0.96      0.96      0.96       809

        accuracy                           0.97      7063
       macro avg       0.97      0.97      0.97      7063
    weighted avg       0.97      0.97      0.97      7063

